In [1]:
! pip install -U -q langchain, chromadb, langchain-core, langchain-gooogle-vertexai gradio, unstructure, google-cloud-storage, langchain-core

ERROR: Invalid requirement: 'langchain,'


In [2]:
! source .venv/bin/activate

In [3]:
# import libraries 
from langchain_google_vertexai import VertexAI
from google.cloud import storage
from langchain import LLMChain

In [4]:
# retrievals 
from langchain_community.document_loaders import GCSDirectoryLoader

In [5]:
BUCKET_NAME = "lloyds-genai24lon-2701-bucket"
PROJECT_ID = "lloyds-genai24lon-2701"

In [6]:
llm = VertexAI(model_name = 'text-bison-32k', max_output_tokens = 256, temperature = 0.1, top_p = 0.8, 
               top_k = 40, verbose = True)

In [7]:
loader = GCSDirectoryLoader(project_name = PROJECT_ID, bucket=BUCKET_NAME, prefix="sample_50/")

In [8]:
documents = loader.load()

In [9]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [11]:
docs = text_splitter.split_documents(documents)
print("splitting...")
print(f"No of Documents = {len(docs)}")

splitting...
No of Documents = 1111


In [12]:
from langchain_google_vertexai import VertexAIEmbeddings

In [13]:
# Structure ingested documents 

In [14]:
REQUESTS_PER_MINUTE = 50

In [15]:
embeddings = VertexAIEmbeddings(requests_per_minute= REQUESTS_PER_MINUTE)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [16]:
# Vector Store

In [17]:
from langchain_community.vectorstores import Chroma

In [18]:
documents_vector_db = Chroma.from_documents(docs, embeddings)

In [42]:
retriever = documents_vector_db.as_retriever(search_type = 'mmr', search_kwargs = {"k":10}, score_threshold = 0.80)

In [43]:
# Retrieval QA Chain 

In [54]:
db2 = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [44]:
from langchain.chains import RetrievalQA

In [45]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever, return_source_documents=True)

In [66]:
result = qa({"query": "what is the weather"})

In [67]:
result

{'query': 'what is the weather',
 'result': ' The provided text does not contain any information about the weather, so I cannot answer this question.',
 'source_documents': [Document(page_content='possible to calculate the rate of investment growth.\n\nThe policy terms allow for an increase in premiums due to increases in mortality\n\ncosts, but I have not assessed whether the increase was fair and reasonable.\n\nReAssure should continue to provide information on Mrs Hs policy based on an\n\nannual growth rate of 6%.\n\nThe new policy that Mrs Hs adviser has obtained a quote for would offer better terms\n\nat a cheaper premium. The quote is evidence that ReAssure are treating new\n\ncustomers more favourably than existing policy holders.\n\nReAssure did not respond to my provisional decision, but they have provided further\n\ninformation that I requested so that I could come to my final decision.What Ive decided  and why\n\nIve considered all the available evidence and arguments to dec

### Gradio 

In [80]:
fail_template = ['No', 'No response', 'contain', 'cannot']

In [81]:
result['result'].split(" ")

['',
 'The',
 'provided',
 'text',
 'does',
 'not',
 'contain',
 'any',
 'information',
 'about',
 'the',
 'weather,',
 'so',
 'I',
 'cannot',
 'answer',
 'this',
 'question.']

In [82]:
fail_result = [any(word in fail_template for word in result['result'].split(" "))]

In [83]:
fail_result

[True]

In [84]:
def chatbot(inputtext):
    result = qa({"query": inputtext})
    
    result = result['result']
    source_url = get_public_url(result['source_documents'][0].metadata['source'])
    file = result['source_documents'][1].metadata['source']
    
    fail_result = [any(word in fail_template for word in result['result'].split(" "))]
    
    if fail_result:
        return result
    else: 
        return result, source_url, file
    
    
    
    

In [49]:
def get_public_url(url): 
    components = url.split("/")
    bucket_name = components[2]
    file_name = components[3]
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.public_url

In [50]:
import gradio as gr

In [85]:
examples = ['What is the context of the first page', 'Did acumen act appropriately with the guidelines',
            'What is the role of the FCA', 'What is your plan for the weekend', 'what is weather']

In [86]:
iface = gr.Interface(fn=chatbot, 
                     inputs = [gr.Textbox(label="Query")], 
                     examples=examples, title = "FOS chatbot", 
                     outputs = [gr.Textbox(label="Response"), 
                               gr.Textbox(label="URL"), gr.Textbox(label = "Cloud Storage URL")])

In [87]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://e4509045d7ac1f58ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/jupyter/.venv/lib/python3.10/site-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "/home/jupyter/.venv/lib/python3.10/site-packages/gradio/route_utils.py", line 252, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/jupyter/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1664, in process_api
    result = await self.call_function(
  File "/home/jupyter/.venv/lib/python3.10/site-packages/gradio/blocks.py", line 1205, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/jupyter/.venv/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/home/jupyter/.venv/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2144, in run_sync_in_worker_thread
    return await future
  File "/home/jupyter/.venv/lib/python3.1